<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BERT/blob/main/ExemplosQuestionAnweringBERT_pt_br.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo de Question Answering(pt-br) usando BERT Transformers by HuggingFace

## **A execução pode ser feita através do menu Ambiente de Execução opção Executar tudo.**

Exemplos **Question Answering(pt-br)** usando **BERT**.

Usando biblioteca Transformer e MCL BERT ajustado para QA.

# 1 Instalação dos pacotes das biblioteca Transformer

Instale a interface pytorch para o BERT by Hugging Face. (Esta biblioteca contém interfaces para outros modelos de linguagem pré-treinados, como o GPT e o GPT-2 da OpenAI.)

Selecionamos a interface pytorch porque ela encontra um bom equilíbrio entre as APIs de alto nível (que são fáceis de usar, mas não fornecem informações sobre como as coisas funcionam) e o código do tensorflow (que contém muitos detalhes, mas muitas vezes nos desvia lições sobre o fluxo tensor, quando o objetivo aqui é o BERT!).

Se você estiver executando esse código no Google Colab, precisará instalar esta biblioteca sempre que se reconectar; a célula a seguir cuidará disso para você.

https://huggingface.co/docs/transformers/installation

In [1]:
# Instala a última versão da biblioteca
#!pip install transformers

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.44.2

# 2 Modelos BERT de QA Disponíveis

https://huggingface.co/models?filter=pt&pipeline_tag=question-answering


* "pucpr/bioBERTpt-squad-v1.1-portuguese"
* "pierreguillou/bert-base-cased-squad-v1.1-portuguese"
* "pierreguillou/bert-large-cased-squad-v1.1-portuguese"
* "mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt"






# 3 Usando Classes do Huggingface



## Carrega o modelo

In [2]:
# Importando as bibliotecas
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

MODELO_BERT = "mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt"
#MODELO_BERT = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"
#MODELO_BERT = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODELO_BERT)

# Carrega o modelo
model = AutoModelForQuestionAnswering.from_pretrained(MODELO_BERT)

tokenizer_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Exemplo 1

Existem três etapas para o QA:
1. tokenizar a entrada
2. obter pontuações do modelo
3. obter a resposta

Essas etapas são discutidas em detalhes no HF [Transformer Notebooks](https://huggingface.co/transformers/notebooks.html).


In [3]:
contexto = r"""
          Pilha é uma estrutura de dados linear que segue uma ordem específica na qual as
          operações são executadas. Um pedido  pode ser UEPS(último a entrar é o primeiro
          a sair) ou PEPS(primeiro a entrar é o primeiro a sair). Em inglês UEPS é
          LIFO(last in first out) e PEPS é FIFO(first in firt out). Existem muitos
          exemplos da vida real de uma pilha. Considere um exemplo de pratos empilhados um
          sobre o outro na cantina. O prato que está no topo é o primeiro a ser removido,
          isto é, o prato que foi colocada na posição mais inferior permanece na pilha
          pelo maior período de tempo. Portanto, pode-se ver simplesmente seguir a ordem
          UEPS/PEPS. Uma fila é uma estrutura linear que segue uma ordem específica na
          qual as operações são executadas. O pedido em um restaurante é o primeiro a
          entrar e o primeiro a sair(PEPS). Um bom exemplo de fila é qualquer fila de
          consumidores para um recurso em que o consumidor que veio primeiro é atendido
          primeiro. A diferença entre pilhas e filas está na remoção. Em uma pilha,
          removemos o item adicionado mais recentemente; em uma fila, removemos o item
          adicionado menos recentemente.
          """

pergunta = "O que é uma pilha?"

# 1. Tokenizando a entrada e o contexto
entrada = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt")

# 2. Obtendo a pontuação do modelo
pontuacao_inicio_resposta, pontuacao_fim_resposta = model(**entrada, return_dict=False)
inicio_resposta = torch.argmax(pontuacao_inicio_resposta)
fim_resposta = torch.argmax(pontuacao_fim_resposta) + 1

# 3. Obtendo a resposta
resposta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entrada["input_ids"][0][inicio_resposta:fim_resposta]))

print(f"Resposta:", resposta)

Resposta: uma estrutura de dados linear que segue uma ordem especifica na qual as operacoes sao executadas


In [4]:
contexto = r"""
          Pilha é uma estrutura de dados linear que segue uma ordem específica na qual as
          operações são executadas. Um pedido  pode ser UEPS(último a entrar é o primeiro
          a sair) ou PEPS(primeiro a entrar é o primeiro a sair). Em inglês UEPS é
          LIFO(last in first out) e PEPS é FIFO(first in firt out). Existem muitos
          exemplos da vida real de uma pilha. Considere um exemplo de pratos empilhados um
          sobre o outro na cantina. O prato que está no topo é o primeiro a ser removido,
          isto é, o prato que foi colocada na posição mais inferior permanece na pilha
          pelo maior período de tempo. Portanto, pode-se ver simplesmente seguir a ordem
          UEPS/PEPS. Uma fila é uma estrutura linear que segue uma ordem específica na
          qual as operações são executadas. O pedido em um restaurante é o primeiro a
          entrar e o primeiro a sair(PEPS). Um bom exemplo de fila é qualquer fila de
          consumidores para um recurso em que o consumidor que veio primeiro é atendido
          primeiro. A diferença entre pilhas e filas está na remoção. Em uma pilha,
          removemos o item adicionado mais recentemente; em uma fila, removemos o item
          adicionado menos recentemente.
          """

pergunta = "Como empilhar um elemento em uma pilha?"

# 1. Tokenizando a entrada e o contexto
entrada = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt")

# 2. Obtendo a pontuação do modelo
pontuacao_inicio_resposta, pontuacao_fim_resposta = model(**entrada, return_dict=False)
inicio_resposta = torch.argmax(pontuacao_inicio_resposta)
fim_resposta = torch.argmax(pontuacao_fim_resposta) + 1

# 3. Obtendo a resposta
resposta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entrada["input_ids"][0][inicio_resposta:fim_resposta]))

print(f"Resposta:", resposta)

Resposta: removemos o item adicionado mais recentemente


In [5]:
contexto = r"""
          Pilha é uma estrutura de dados linear que segue uma ordem específica na qual as
          operações são executadas. Um pedido  pode ser UEPS(último a entrar é o primeiro
          a sair) ou PEPS(primeiro a entrar é o primeiro a sair). Em inglês UEPS é
          LIFO(last in first out) e PEPS é FIFO(first in firt out). Existem muitos
          exemplos da vida real de uma pilha. Considere um exemplo de pratos empilhados um
          sobre o outro na cantina. O prato que está no topo é o primeiro a ser removido,
          isto é, o prato que foi colocada na posição mais inferior permanece na pilha
          pelo maior período de tempo. Portanto, pode-se ver simplesmente seguir a ordem
          UEPS/PEPS. Uma fila é uma estrutura linear que segue uma ordem específica na
          qual as operações são executadas. O pedido em um restaurante é o primeiro a
          entrar e o primeiro a sair(PEPS). Um bom exemplo de fila é qualquer fila de
          consumidores para um recurso em que o consumidor que veio primeiro é atendido
          primeiro. A diferença entre pilhas e filas está na remoção. Em uma pilha,
          removemos o item adicionado mais recentemente; em uma fila, removemos o item
          adicionado menos recentemente.
          """

pergunta = "Como empilhar um elemento em uma fila?"

# 1. Tokenizando a entrada e o contexto
entrada = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt")

# 2. Obtendo a pontuação do modelo
pontuacao_inicio_resposta, pontuacao_fim_resposta = model(**entrada, return_dict=False)
fim_resposta = torch.argmax(pontuacao_fim_resposta) + 1

# 3. Obtendo a resposta
resposta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entrada["input_ids"][0][inicio_resposta:fim_resposta]))

print(f"Resposta:", resposta)

Resposta: , removemos o item adicionado mais recentemente ; em uma fila, removemos o item adicionado menos recentemente


## Exemplo 2


In [6]:
contexto = r"""
          A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19,
          uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2).
          A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China,
          em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano.
          Acredita-se que o vírus tenha uma origem zoonótica, porque os primeiros casos confirmados
          tinham principalmente ligações ao Mercado Atacadista de Frutos do Mar de Huanan, que também vendia animais vivos.
          Em 11 de março de 2020, a Organização Mundial da Saúde declarou o surto uma pandemia. Até 8 de fevereiro de 2021,
          pelo menos 105 743 102 casos da doença foram confirmados em pelo menos 191 países e territórios,
          com cerca de 2 308 943 mortes e 58 851 440 pessoas curadas.
          """

pergunta = "Quando começou a pandemia de Covid-19 no mundo?"

# 1. Tokenizando a entrada e o contexto
entrada = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt")

# 2. Obtendo a pontuação do modelo
pontuacao_inicio_resposta, pontuacao_fim_resposta = model(**entrada, return_dict=False)
inicio_resposta = torch.argmax(pontuacao_inicio_resposta)
fim_resposta = torch.argmax(pontuacao_fim_resposta) + 1

# 3. Obtendo a resposta
resposta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entrada["input_ids"][0][inicio_resposta:fim_resposta]))

print(f"Resposta:", resposta)

Resposta: 1 de dezembro de 2019


## Exemplo 3

In [7]:
contexto = r"""No dia 22 de abril comemoramos o descobrimento do Brasil. Na mesma data,
            só que no ano de 1.500, Pedro Álvares Cabral chegava às terras de "Vera Cruz",
            que mais tarde seriam chamadas de Brasil.
            Ele chegou próximo à região de Porto Seguro, no estado da Bahia.
            Pedro Álvares Cabral não foi o primeiro português a pisar
            em solo brasileiro. Na verdade, quem fez isso foi Duarte Pacheco Pereira, um navegador militar.
            Em 1498 ele foi designado por D. Manuel I para uma expedição secreta para reconhecer as zonas situadas além da marcação de Tordesilhas.
            A expedição dele partiu do arquipélago de Cabo Verde e chegou em algum ponto da costa entre o Maranhão e o Pará. A partir desse ponto,
            Pereira seguido pela costa Norte, até a foz do Rio Amazonas e a Ilha de Marajó."""

pergunta = "Quem descobriu o Brasil?"

# 1. Tokenizando a entrada e o contexto
entrada = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt")

# 2. Obtendo a pontuação do modelo
pontuacao_inicio_resposta, pontuacao_fim_resposta = model(**entrada, return_dict=False)
inicio_resposta = torch.argmax(pontuacao_inicio_resposta)
fim_resposta = torch.argmax(pontuacao_fim_resposta) + 1

# 3. Obtendo a resposta
resposta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entrada["input_ids"][0][inicio_resposta:fim_resposta]))

print(f"Resposta:", resposta)

Resposta: pedro alvares cabral


# 4 Usando Pipeline do Huggingface

## Carrega o pipeline

In [8]:
# Importando as bibliotecas
import transformers
from transformers import pipeline

In [9]:
# Carrega o pipeline
model_qa = pipeline("question-answering",
                    model="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt",
                    tokenizer="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt")

Some weights of the model checkpoint at mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check thi

In [10]:
# Importando as bibliotecas
#import transformers
#from transformers import QuestionAnsweringPipeline

# model_qa = QuestionAnsweringPipeline(model="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt", tokenizer="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt")

## Exemplo 1

In [11]:
contexto = r"""
          Pilha é uma estrutura de dados linear que segue uma ordem específica na qual as
          operações são executadas. Um pedido  pode ser UEPS(último a entrar é o primeiro
          a sair) ou PEPS(primeiro a entrar é o primeiro a sair). Em inglês UEPS é
          LIFO(last in first out) e PEPS é FIFO(first in firt out). Existem muitos
          exemplos da vida real de uma pilha. Considere um exemplo de pratos empilhados um
          sobre o outro na cantina. O prato que está no topo é o primeiro a ser removido,
          isto é, o prato que foi colocada na posição mais inferior permanece na pilha
          pelo maior período de tempo. Portanto, pode-se ver simplesmente seguir a ordem
          UEPS/PEPS. Uma fila é uma estrutura linear que segue uma ordem específica na
          qual as operações são executadas. O pedido em um restaurante é o primeiro a
          entrar e o primeiro a sair(PEPS). Um bom exemplo de fila é qualquer fila de
          consumidores para um recurso em que o consumidor que veio primeiro é atendido
          primeiro. A diferença entre pilhas e filas está na remoção. Em uma pilha,
          removemos o item adicionado mais recentemente; em uma fila, removemos o item
          adicionado menos recentemente.
          """

pergunta = "O que é uma pilha?"

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado["answer"], ", pontuação: ",  round(resultado["score"], 4),  ", início: ", resultado["start"],  ", fim: ", resultado["end"])

 Resposta QA:  uma estrutura de dados linear , pontuação:  0.0005 , início:  19 , fim:  48


In [12]:
contexto = r"""
          Pilha é uma estrutura de dados linear que segue uma ordem específica na qual as
          operações são executadas. Um pedido  pode ser UEPS(último a entrar é o primeiro
          a sair) ou PEPS(primeiro a entrar é o primeiro a sair). Em inglês UEPS é
          LIFO(last in first out) e PEPS é FIFO(first in firt out). Existem muitos
          exemplos da vida real de uma pilha. Considere um exemplo de pratos empilhados um
          sobre o outro na cantina. O prato que está no topo é o primeiro a ser removido,
          isto é, o prato que foi colocada na posição mais inferior permanece na pilha
          pelo maior período de tempo. Portanto, pode-se ver simplesmente seguir a ordem
          UEPS/PEPS. Uma fila é uma estrutura linear que segue uma ordem específica na
          qual as operações são executadas. O pedido em um restaurante é o primeiro a
          entrar e o primeiro a sair(PEPS). Um bom exemplo de fila é qualquer fila de
          consumidores para um recurso em que o consumidor que veio primeiro é atendido
          primeiro. A diferença entre pilhas e filas está na remoção. Em uma pilha,
          removemos o item adicionado mais recentemente; em uma fila, removemos o item
          adicionado menos recentemente.
          """

pergunta = "Como empilhar um elemento em uma pilha?"

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado["answer"], ", pontuação: ",  round(resultado["score"], 4),  ", início: ", resultado["start"],  ", fim: ", resultado["end"])

 Resposta QA:  removemos o item adicionado mais recentemente , pontuação:  0.1676 , início:  1145 , fim:  1190


In [13]:
contexto = r"""
          Pilha é uma estrutura de dados linear que segue uma ordem específica na qual as
          operações são executadas. Um pedido  pode ser UEPS(último a entrar é o primeiro
          a sair) ou PEPS(primeiro a entrar é o primeiro a sair). Em inglês UEPS é
          LIFO(last in first out) e PEPS é FIFO(first in firt out). Existem muitos
          exemplos da vida real de uma pilha. Considere um exemplo de pratos empilhados um
          sobre o outro na cantina. O prato que está no topo é o primeiro a ser removido,
          isto é, o prato que foi colocada na posição mais inferior permanece na pilha
          pelo maior período de tempo. Portanto, pode-se ver simplesmente seguir a ordem
          UEPS/PEPS. Uma fila é uma estrutura linear que segue uma ordem específica na
          qual as operações são executadas. O pedido em um restaurante é o primeiro a
          entrar e o primeiro a sair(PEPS). Um bom exemplo de fila é qualquer fila de
          consumidores para um recurso em que o consumidor que veio primeiro é atendido
          primeiro. A diferença entre pilhas e filas está na remoção. Em uma pilha,
          removemos o item adicionado mais recentemente; em uma fila, removemos o item
          adicionado menos recentemente.
          """

pergunta = "Como empilhar um elemento em uma fila?"

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado["answer"], ", pontuação: ",  round(resultado["score"], 4),  ", início: ", resultado["start"],  ", fim: ", resultado["end"])

 Resposta QA:  removemos o item
          adicionado menos recentemente , pontuação:  0.5753 , início:  1205 , fim:  1261


In [14]:
contexto = r"""
          Pilha é uma estrutura de dados linear que segue uma ordem específica na qual as
          operações são executadas. Um pedido  pode ser UEPS(último a entrar é o primeiro
          a sair) ou PEPS(primeiro a entrar é o primeiro a sair). Em inglês UEPS é
          LIFO(last in first out) e PEPS é FIFO(first in firt out). Existem muitos
          exemplos da vida real de uma pilha. Considere um exemplo de pratos empilhados um
          sobre o outro na cantina. O prato que está no topo é o primeiro a ser removido,
          isto é, o prato que foi colocada na posição mais inferior permanece na pilha
          pelo maior período de tempo. Portanto, pode-se ver simplesmente seguir a ordem
          UEPS/PEPS. Uma fila é uma estrutura linear que segue uma ordem específica na
          qual as operações são executadas. O pedido em um restaurante é o primeiro a
          entrar e o primeiro a sair(PEPS). Um bom exemplo de fila é qualquer fila de
          consumidores para um recurso em que o consumidor que veio primeiro é atendido
          primeiro. A diferença entre pilhas e filas está na remoção. Em uma pilha,
          removemos o item adicionado mais recentemente; em uma fila, removemos o item
          adicionado menos recentemente.
          """

pergunta = "Como enfileirar um elemento em uma fila?"

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado["answer"], ", pontuação: ",  round(resultado["score"], 4),  ", início: ", resultado["start"],  ", fim: ", resultado["end"])

 Resposta QA:  removemos o item
          adicionado menos recentemente , pontuação:  0.5103 , início:  1205 , fim:  1261


## Exemplo 2

In [15]:
contexto = r"""
          A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19,
          uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2).
          A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China,
          em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano.
          Acredita-se que o vírus tenha uma origem zoonótica, porque os primeiros casos confirmados
          tinham principalmente ligações ao Mercado Atacadista de Frutos do Mar de Huanan, que também vendia animais vivos.
          Em 11 de março de 2020, a Organização Mundial da Saúde declarou o surto uma pandemia. Até 8 de fevereiro de 2021,
          pelo menos 105 743 102 casos da doença foram confirmados em pelo menos 191 países e territórios,
          com cerca de 2 308 943 mortes e 58 851 440 pessoas curadas.
          """

pergunta = "Quando começou a pandemia de Covid-19 no mundo?"

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado["answer"], ", pontuação: ",  round(resultado["score"], 4),  ", início: ", resultado["start"],  ", fim: ", resultado["end"])

 Resposta QA:  1 de dezembro de 2019 , pontuação:  0.695 , início:  368 , fim:  389


## Exemplo 3

In [16]:
pergunta = "Quem descobriu o Brasil?"

contexto = """No dia 22 de abril comemoramos o descobrimento do Brasil. Na mesma data,
            só que no ano de 1.500, Pedro Álvares Cabral chegava às terras de "Vera Cruz",
            que mais tarde seriam chamadas de Brasil.
            Ele chegou próximo à região de Porto Seguro, no estado da Bahia.
            Pedro Álvares Cabral não foi o primeiro português a pisar
            em solo brasileiro. Na verdade, quem fez isso foi Duarte Pacheco Pereira, um navegador militar.
            Em 1498 ele foi designado por D. Manuel I para uma expedição secreta para reconhecer as zonas situadas além da marcação de Tordesilhas.
            A expedição dele partiu do arquipélago de Cabo Verde e chegou em algum ponto da costa entre o Maranhão e o Pará. A partir desse ponto,
            Pereira seguido pela costa Norte, até a foz do Rio Amazonas e a Ilha de Marajó."""

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado["answer"], ", pontuação: ",  round(resultado["score"], 4),  ", início: ", resultado["start"],  ", fim: ", resultado["end"])

 Resposta QA:  Pedro Álvares Cabral , pontuação:  0.9999 , início:  110 , fim:  130


# 5 Respondendo várias perguntas de um conjunto de dados

## Instalando a biblioteca Dataset

https://huggingface.co/docs/datasets/installation.html

In [17]:
!pip install datasets==3.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00


In [18]:
# Import da biblioteca
from huggingface_hub import list_datasets

# Carregando a lista do conjunto de dados
datasets_list = list_datasets()

# Imprime o tamanho do conjunto de dados
#print(len(datasets_list))
cont = 0
for dataset in datasets_list:
    cont = cont + 1
print(cont)

# for dataset in datasets_list:
#     print(dataset)

232230


## Carregando um conjunto de dados

Carregando o SQUAD 1.0 traduzido.

Lista dos conjunto de dados em português.
https://huggingface.co/datasets?filter=languages:pt

In [19]:
# Import da biblioteca
from datasets import load_dataset

# Carrega o conjunto de dados
datasets = load_dataset("squad_v2")

print(datasets)

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


## Respondendo um conjunto de dados de perguntas

In [20]:
# Importando as bibliotecas
import transformers
from transformers import pipeline

In [21]:
# Carrega o pipeline
model_qa = pipeline("question-answering",
                    model="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt",
                    tokenizer="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt")

Some weights of the model checkpoint at mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
# Percorre o conjunto de dados
for i, pergunta in enumerate(datasets["validation"]):
  # Responde somente 10 perguntas
  if i < 10:
    print("\nPergunta:", pergunta["question"])
    print("Contexto:",pergunta["context"])
    print("       Respostas esperada:",pergunta["answers"])
    for j, resposta in enumerate(pergunta["answers"]["text"]):
        print("       Resposta esperada:",resposta)

    resultado = model_qa(question=pergunta["question"], context=pergunta["context"])

    print("      Resposta QA: ", resultado["answer"], ", pontuação: ",  round(resultado["score"], 4),  ", início: ", resultado["start"],  ", fim: ", resultado["end"])



Pergunta: In what country is Normandy located?
Contexto: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
       Respostas esperada: {'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}
       Resposta esperada: France
       Resposta esperada: France
       Resposta e